# Day 3 


In [2]:
import gzip
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import Word2Vec, LdaMulticore

from gensim.corpora import Dictionary 

import numpy as np 
import pyLDAvis
from pyLDAvis import gensim



In [3]:
ls data

reviews_data.txt


In [7]:
with open('data/reviews_data.txt', 'rb') as f:
    for i, line in enumerate(f):
        print(i, line)
        print('---------------')
        if i > 2: break

0 b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in 

In [8]:
def read_sentences(filename):    
    with open(filename, 'rb') as f:
        for line in f:
            yield line
            
sentences = read_sentences('data/reviews_data.txt')

In [17]:
txt = '\tStylish clean and reasonable value poor location\tI am glad to be the first person to post photos of this hotel. Key points about this property:- its a starwood property, much cheape'
simple_preprocess(txt)

['stylish',
 'clean',
 'and',
 'reasonable',
 'value',
 'poor',
 'location',
 'am',
 'glad',
 'to',
 'be',
 'the',
 'first',
 'person',
 'to',
 'post',
 'photos',
 'of',
 'this',
 'hotel',
 'key',
 'points',
 'about',
 'this',
 'property',
 'its',
 'starwood',
 'property',
 'much',
 'cheape']

In [13]:
 STOPWORDS

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [15]:
def preprocessing(sentence):
    return [word for word in simple_preprocess(sentence) \
            if word not in STOPWORDS]

In [18]:
preprocessing(txt)

['stylish',
 'clean',
 'reasonable',
 'value',
 'poor',
 'location',
 'glad',
 'person',
 'post',
 'photos',
 'hotel',
 'key',
 'points',
 'property',
 'starwood',
 'property',
 'cheape']

In [27]:
def read_sentences(filename):    
    with open(filename, 'rb') as f:
        for line in f:
            yield preprocessing(line)
            
%time sentences = list(read_sentences('data/reviews_data.txt'))

CPU times: user 1min 29s, sys: 2.15 s, total: 1min 32s
Wall time: 1min 33s


In [28]:
len(sentences)

255404

In [29]:
%time model = Word2Vec(sentences, size=100, window=5, min_count=2)

CPU times: user 5min 28s, sys: 3.06 s, total: 5min 31s
Wall time: 2min 19s


In [30]:
#model = Word2vec.load('src')

## Vector representation

In [31]:
model.wv.most_similar('good')

[('decent', 0.8346644639968872),
 ('great', 0.818388044834137),
 ('excellent', 0.8070321083068848),
 ('ok', 0.6500048637390137),
 ('fair', 0.6469687223434448),
 ('terrific', 0.6353988647460938),
 ('reasonable', 0.6227473020553589),
 ('nice', 0.6175752878189087),
 ('fantastic', 0.6081993579864502),
 ('average', 0.6055147647857666)]

In [33]:
model.wv.most_similar('bad')

[('terrible', 0.7200518250465393),
 ('horrible', 0.6702731847763062),
 ('awful', 0.6416341066360474),
 ('negative', 0.6114100217819214),
 ('poor', 0.6076475381851196),
 ('positive', 0.5832971334457397),
 ('okay', 0.578356146812439),
 ('worse', 0.5649154782295227),
 ('lousy', 0.5605853796005249),
 ('unpleasant', 0.5504626035690308)]

In [34]:
model.wv.most_similar(positive=['bad', 'terrible'], negative=['good'])

[('horrible', 0.8401877880096436),
 ('awful', 0.8090313673019409),
 ('dreadful', 0.7066207528114319),
 ('horrendous', 0.6820273399353027),
 ('miserable', 0.6670020222663879),
 ('shocking', 0.6654716730117798),
 ('horrid', 0.6648236513137817),
 ('appalling', 0.6647194623947144),
 ('horrific', 0.6600464582443237),
 ('nasty', 0.6528175473213196)]

In [37]:
model.wv.most_similar('reception')

[('desk', 0.7000306844711304),
 ('recption', 0.6710910797119141),
 ('receptionist', 0.6237592697143555),
 ('receiption', 0.6210138201713562),
 ('receptionists', 0.6121381521224976),
 ('frontdesk', 0.5747754573822021),
 ('recpetion', 0.5711360573768616),
 ('porter', 0.5638704299926758),
 ('apologised', 0.5525252819061279),
 ('bellboy', 0.5493656396865845)]

## Topic modelling - LDA


In [38]:
sentences_light  = np.random.permutation(sentences)

In [39]:
sentences_light = sentences_light[:1000]

In [40]:
%time dictionary = Dictionary(sentences_light)

CPU times: user 153 ms, sys: 7.13 ms, total: 161 ms
Wall time: 158 ms


In [42]:
list(dictionary.items())

[(0, 'bar'),
 (1, 'bed'),
 (2, 'cells'),
 (3, 'courthouse'),
 (4, 'decor'),
 (5, 'end'),
 (6, 'excellent'),
 (7, 'experience'),
 (8, 'fans'),
 (9, 'going'),
 (10, 'good'),
 (11, 'hotel'),
 (12, 'ideally'),
 (13, 'larger'),
 (14, 'll'),
 (15, 'location'),
 (16, 'needed'),
 (17, 'numerous'),
 (18, 'oct'),
 (19, 'old'),
 (20, 'original'),
 (21, 'originally'),
 (22, 'picked'),
 (23, 'placed'),
 (24, 'queensize'),
 (25, 'received'),
 (26, 'restaurants'),
 (27, 'ro'),
 (28, 'room'),
 (29, 'sky'),
 (30, 'small'),
 (31, 'sports'),
 (32, 'tad'),
 (33, 'taking'),
 (34, 'taxi'),
 (35, 'think'),
 (36, 'time'),
 (37, 'try'),
 (38, 'tv'),
 (39, 'value'),
 (40, 'welcome'),
 (41, 'west'),
 (42, 'able'),
 (43, 'anticipated'),
 (44, 'ben'),
 (45, 'big'),
 (46, 'bridge'),
 (47, 'bridges'),
 (48, 'centrally'),
 (49, 'city'),
 (50, 'county'),
 (51, 'crossing'),
 (52, 'directions'),
 (53, 'easy'),
 (54, 'eat'),
 (55, 'england'),
 (56, 'exchange'),
 (57, 'eye'),
 (58, 'feb'),
 (59, 'ferris'),
 (60, 'fine'),


In [43]:
len(dictionary)

8777

In [44]:
%time bow_corpus = [dictionary.doc2bow(s) for s in sentences_light]

CPU times: user 107 ms, sys: 8.34 ms, total: 115 ms
Wall time: 115 ms


In [45]:
dictionary.doc2bow(['car', 'car', 'hotel'])

[(11, 1), (642, 2)]

In [49]:
dictionary.token2id['hotel']

11

In [55]:
%time lda_model = LdaMulticore(bow_corpus, id2word=dictionary, num_topics=10, passes=20, workers=8)

CPU times: user 52.3 s, sys: 3.77 s, total: 56.1 s
Wall time: 56.4 s


In [57]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.014*"room" + 0.013*"hotel" + 0.005*"day" + 0.005*"booked" + 0.004*"staff" + 0.004*"nov" + 0.004*"aug" + 0.004*"horrible" + 0.004*"stayed" + 0.004*"wanted"
Topic: 1 
Words: 0.033*"hotel" + 0.023*"room" + 0.012*"location" + 0.012*"good" + 0.011*"great" + 0.010*"clean" + 0.010*"staff" + 0.009*"stay" + 0.008*"nice" + 0.008*"breakfast"
Topic: 2 
Words: 0.013*"hotel" + 0.011*"stay" + 0.010*"room" + 0.009*"quot" + 0.005*"location" + 0.005*"time" + 0.005*"good" + 0.005*"city" + 0.004*"th" + 0.004*"stayed"
Topic: 3 
Words: 0.039*"hotel" + 0.020*"great" + 0.019*"room" + 0.014*"stay" + 0.011*"staff" + 0.008*"location" + 0.008*"rooms" + 0.008*"nice" + 0.007*"stayed" + 0.006*"clean"
Topic: 4 
Words: 0.031*"room" + 0.014*"hotel" + 0.011*"quot" + 0.008*"night" + 0.008*"rooms" + 0.007*"staff" + 0.006*"desk" + 0.006*"time" + 0.006*"stay" + 0.006*"day"
Topic: 5 
Words: 0.022*"hotel" + 0.012*"room" + 0.011*"great" + 0.010*"good" + 0.009*"stay" + 0.009*"rooms" + 0.007*"nice" + 0.007*"lo

## Topic Visualization

In [56]:
lda_vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(lda_vis)

/anaconda3/envs/env-dwc4/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
